<a href="https://colab.research.google.com/github/kleczekr/tolkenizer/blob/master/gensim_topics_transformations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import logging
from collections import defaultdict
from gensim import corpora
from gensim import models
import os
import tempfile
%matplotlib inline

In [2]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
documents = [
    "Human machine interface for lab abc computer applications",
    "A survey of user opinion of computer system response time",
    "The EPS user interface management system",
    "System and human system engineering testing of EPS",
    "Relation of user perceived response time to error measurement",
    "The generation of random binary unordered trees",
    "The intersection graph of paths in trees",
    "Graph minors IV Widths of trees and well quasi ordering",
    "Graph minors A survey",
]
# remove common words and tokenize
stoplist = set('for a of the and to in'.split())
texts = [
    [word for word in document.lower().split() if word not in stoplist]
    for document in documents
]

# remove words that appear only once
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [
    [token for token in text if frequency[token] > 1]
    for text in texts
]

dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

2020-07-27 22:02:38,143 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-07-27 22:02:38,152 : INFO : built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)


In [4]:
tfidf = models.TfidfModel(corpus)  # step 1 -- initialize a model

2020-07-27 22:02:38,162 : INFO : collecting document frequencies
2020-07-27 22:02:38,163 : INFO : PROGRESS: processing document #0
2020-07-27 22:02:38,165 : INFO : calculating IDF weights for 9 documents and 11 features (28 matrix non-zeros)


In [5]:
doc_bow = [(0, 1), (1, 1)]
print(tfidf[doc_bow])  # step 2 -- use the model to transform vectors

[(0, 0.7071067811865476), (1, 0.7071067811865476)]


In [6]:
# transform the entire corpus; conversions are done on-the fly, during subsequent iteration
corpus_tfidf = tfidf[corpus]
for doc in corpus_tfidf:
    print(doc)

[(0, 0.5773502691896257), (1, 0.5773502691896257), (2, 0.5773502691896257)]
[(0, 0.44424552527467476), (3, 0.44424552527467476), (4, 0.44424552527467476), (5, 0.3244870206138555), (6, 0.44424552527467476), (7, 0.3244870206138555)]
[(2, 0.5710059809418182), (5, 0.4170757362022777), (7, 0.4170757362022777), (8, 0.5710059809418182)]
[(1, 0.49182558987264147), (5, 0.7184811607083769), (8, 0.49182558987264147)]
[(3, 0.6282580468670046), (6, 0.6282580468670046), (7, 0.45889394536615247)]
[(9, 1.0)]
[(9, 0.7071067811865475), (10, 0.7071067811865475)]
[(9, 0.5080429008916749), (10, 0.5080429008916749), (11, 0.695546419520037)]
[(4, 0.6282580468670046), (10, 0.45889394536615247), (11, 0.6282580468670046)]


In [7]:
# the following serializes transformations---one on top of the other
lsi_model = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=2)  # initialize an LSI transformation
corpus_lsi = lsi_model[corpus_tfidf]  # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi

2020-07-27 22:02:38,197 : INFO : using serial LSI version on this node
2020-07-27 22:02:38,198 : INFO : updating model with new documents
2020-07-27 22:02:38,200 : INFO : preparing a new chunk of documents
2020-07-27 22:02:38,201 : INFO : using 100 extra samples and 2 power iterations
2020-07-27 22:02:38,203 : INFO : 1st phase: constructing (12, 102) action matrix
2020-07-27 22:02:38,205 : INFO : orthonormalizing (12, 102) action matrix
2020-07-27 22:02:38,208 : INFO : 2nd phase: running dense svd on (12, 9) matrix
2020-07-27 22:02:38,209 : INFO : computing the final decomposition
2020-07-27 22:02:38,211 : INFO : keeping 2 factors (discarding 47.565% of energy spectrum)
2020-07-27 22:02:38,212 : INFO : processed documents up to #9
2020-07-27 22:02:38,214 : INFO : topic #0(1.594): 0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"response" + 0.060*"time" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"
2020-07-27 22:02:38,215 : INFO : topic #

In [8]:
lsi_model.print_topics(2)

2020-07-27 22:02:38,224 : INFO : topic #0(1.594): 0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"response" + 0.060*"time" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"
2020-07-27 22:02:38,227 : INFO : topic #1(1.476): -0.460*"system" + -0.373*"user" + -0.332*"eps" + -0.328*"interface" + -0.320*"time" + -0.320*"response" + -0.293*"computer" + -0.280*"human" + -0.171*"survey" + 0.161*"trees"


[(0,
  '0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"response" + 0.060*"time" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"'),
 (1,
  '-0.460*"system" + -0.373*"user" + -0.332*"eps" + -0.328*"interface" + -0.320*"time" + -0.320*"response" + -0.293*"computer" + -0.280*"human" + -0.171*"survey" + 0.161*"trees"')]

In [9]:
# both bow->tfidf and tfidf->lsi transformations are actually executed here, on the fly
for doc, as_text in zip(corpus_lsi, documents):
    print(doc, as_text)

[(0, 0.06600783396090391), (1, -0.520070330636185)] Human machine interface for lab abc computer applications
[(0, 0.19667592859142505), (1, -0.7609563167700043)] A survey of user opinion of computer system response time
[(0, 0.08992639972446373), (1, -0.7241860626752509)] The EPS user interface management system
[(0, 0.0758584765217807), (1, -0.632055158600343)] System and human system engineering testing of EPS
[(0, 0.10150299184980106), (1, -0.5737308483002952)] Relation of user perceived response time to error measurement
[(0, 0.7032108939378315), (1, 0.1611518021402575)] The generation of random binary unordered trees
[(0, 0.8774787673119836), (1, 0.16758906864659368)] The intersection graph of paths in trees
[(0, 0.9098624686818582), (1, 0.14086553628718976)] Graph minors IV Widths of trees and well quasi ordering
[(0, 0.6165825350569285), (1, -0.053929075663893745)] Graph minors A survey


In [10]:
with tempfile.NamedTemporaryFile(prefix='model-', suffix='.lsi', delete=False) as tmp:
    lsi_model.save(tmp.name)  # same for tfidf, lda, ...

loaded_lsi_model = models.LsiModel.load(tmp.name)

os.unlink(tmp.name)

2020-07-27 22:02:38,255 : INFO : saving Projection object under /tmp/model-15xjtn1l.lsi.projection, separately None
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
2020-07-27 22:02:38,259 : INFO : saved /tmp/model-15xjtn1l.lsi.projection
2020-07-27 22:02:38,260 : INFO : saving LsiModel object under /tmp/model-15xjtn1l.lsi, separately None
2020-07-27 22:02:38,261 : INFO : not storing attribute projection
2020-07-27 22:02:38,262 : INFO : not storing attribute dispatcher
2020-07-27 22:02:38,264 : INFO : saved /tmp/model-15xjtn1l.lsi
2020-07-27 22:02:38,266 : INFO : loading LsiModel object from /tmp/model-15xjtn1l.lsi
2020-07-27 22:02:38,267 : INFO : loading id2word recursively from /tmp/m

Available transformations:

In [11]:
# tf-ldf:
model = models.TfidfModel(corpus, normalize=True)

2020-07-27 22:02:38,281 : INFO : collecting document frequencies
2020-07-27 22:02:38,283 : INFO : PROGRESS: processing document #0
2020-07-27 22:02:38,284 : INFO : calculating IDF weights for 9 documents and 11 features (28 matrix non-zeros)


In [12]:
# latent semantic indexing
#model = models.LsiModel(tfidf_corpus, id2word=dictionary, num_topics=300)

In [13]:
#model.add_documents(another_tfidf_corpus)  # now LSI has been trained on tfidf_corpus + another_tfidf_corpus
#lsi_vec = model[tfidf_vec]  # convert some new document into the LSI space, without affecting the model

#model.add_documents(more_documents)  # tfidf_corpus + another_tfidf_corpus + more_documents
#lsi_vec = model[tfidf_vec]

In [14]:
# random projections
#model = models.RpModel(tfidf_corpus, num_topics=500)

In [15]:
# latent dirichlet allocation
#model = models.LdaModel(corpus, id2word=dictionary, num_topics=100)

In [16]:
# hierarchical dirichlet process
#model = models.HdpModel(corpus, id2word=dictionary)